In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from model.MatrixTransfer import OracleTransReg
from model.utils import batch_mat_prod
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

## Transfer learning under known $\mathcal{A}$

Code for Section 4 Simulation Study.

In [2]:
# generate samples and labels for regression problems
def regression_data(B : np.ndarray, W : list, n0 : int=100, nk : int=100):
    K = len(W)
    p1, p2 = B.shape[0], B.shape[1]
    X_target = np.random.randn(n0,p1,p2)
    y_target = batch_mat_prod(X_target,B)
    X_source, y_source = [], []
    for k in range(K):
        X_s = np.random.randn(nk,p1,p2)
        y_s = batch_mat_prod(X_s,W[k])
        X_source.append(X_s.copy())
        y_source.append(y_s.copy())
    return X_target, X_source, y_target, y_source

In [3]:
# generate samples and labels for binary logistic problems
def binary_logistic_data(B : np.ndarray, W : list, n0 : int=100, nk : int=100):
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))
    K = len(W)
    p1, p2 = B.shape[0], B.shape[1]
    X_target = np.random.randn(n0,p1,p2)
    logits = batch_mat_prod(X_target,B)
    y_target = np.random.binomial(1,sigmoid(logits),size=n0)
    X_source, y_source = [], []
    for k in range(K):
        X_s = np.random.randn(nk,p1,p2)
        logits_t = batch_mat_prod(X_s,W[k])
        y_s = np.random.binomial(1,sigmoid(logits_t),size=nk)
        X_source.append(X_s.copy())
        y_source.append(y_s.copy())
    return X_target, X_source, y_target, y_source

### Under Setting $\mathcal{A}_h^{vec}$

In [4]:
def GenerateCoef_vec(R : int = 5, s : float=0.50, p1 : int=64, p2 : int=64, h : float=0, K : int=10):
    """
    Parameters
    ----------
    R : int, default = `5`
        The rank of the matrix coef.
    s : float, default = `0.5`
        The sparsity of the matrix coef.
    p1, p2 : int, default = `64`
        THe height and width of the coef.
    h : float, default = `0`
        The deviation level of source domain and target domain.
    K : int, defualt = `10`
        THe number of the imformative source domain.

    Return
    ----------
    B : np.ndarray
        The true coef of target domain with shape `(p1,p2)`
    W : list
        The list of source domians' coef
    """
    # generate true coef for target domain
    p = np.sqrt(1 - (1-s)**(1/R))
    B1 = np.random.binomial(1,p,size=(p1,R))
    B2 = np.random.binomial(1,p,size=(p2,R))
    E = np.identity(R)
    B = B1 @ E @ B2.T
    # generate source domian coef
    W = []
    for k in range(K):
        # D_{i,j} in {-1,1} randomly with equal probability
        D = np.random.binomial(1,0.5,size=(p1,p2)) * 2 - 1
        Wk = B + (h/(p1*p2)) * D
        W.append(Wk.copy())
    return B, W

In [5]:
def CreateModel(model : str, task : str="regression", max_steps : int=200,**kwargs):
    if model == "naive-Lasso":
        return OracleTransReg(task=task,penalty_debias="lasso",max_steps=max_steps,**kwargs)
    elif model == "naive-Nuclear":
        return OracleTransReg(task=task,penalty_debias="nuclear",max_steps=max_steps,**kwargs)
    elif model == "A-Lasso-Lasso":
        return OracleTransReg(task=task,penalty_transfer="lasso",penalty_debias="lasso",max_steps=max_steps,**kwargs)
    elif model == "A-Nuclear-Lasso":
        return OracleTransReg(task=task,penalty_transfer="nuclear",penalty_debias="lasso",max_steps=max_steps,**kwargs)
    elif model == "A-Nuclear-Nuclear":
        return OracleTransReg(task=task,penalty_transfer="nuclear",penalty_debias="nuclear",max_steps=max_steps,**kwargs)

In [6]:
# init results
models = ["naive-Lasso","naive-Nuclear","A-Lasso-Lasso","A-Nuclear-Lasso","A-Nuclear-Nuclear"]
result = pd.DataFrame(data=None,columns=["h","K","repeat"] + models)
h_list, Kmax, replicate = [200,500], 10, 100
# show progress
pbar = tqdm(total=len(h_list)*Kmax*replicate,ncols=120)
for h in h_list:
    for K in range(1,Kmax+1):
        optimal_lambda = {}
        for repeat in range(replicate):
            B, W = GenerateCoef_vec(h=h,K=K)
            X_target, X_source, y_target, y_source = regression_data(B=B,W=W)
            line = [h,K,repeat+1]
            # update progress
            pbar.set_description("Fitting on h: %4d, K: %2d, round: %3d"%(h,K,repeat+1))
            for m in models:
                A_set = None if "naive" in m else list(range(K))
                if repeat == 0:
                    model = CreateModel(model=m)
                    best_lambda_transfer, best_lambda_debias =\
                        model.tuning(Xt=X_target,yt=y_target,Xa=X_source,ya=y_source,A=A_set)
                    optimal_lambda["trasnfer_%s"%(m)] = best_lambda_transfer
                    optimal_lambda["debias_%s"%(m)] = best_lambda_debias
                else:
                    model = CreateModel(model=m,
                        lambda_transfer=optimal_lambda["trasnfer_%s"%(m)],
                        lambda_debias=optimal_lambda["debias_%s"%(m)])
                    model.fit(Xt=X_target,yt=y_target,Xa=X_source,ya=y_source,A=A_set)
                err = np.linalg.norm(B - model.coef_,ord="fro") 
                line.append(err)
            pbar.update(1)
            result.loc[len(result),:] = line
pbar.clear()
pbar.close()

Fitting on h:  200, K:  9, round:  18:  41%|███████████████▉                       | 817/2000 [55:33<2:52:27,  8.75s/it]

In [8]:
result.to_csc("./results/simulation_sec4_knownA_vec_linear.csv",index=False)

,h,K,naive-Lasso,naive-Nuclear,A-Lasso-Lasso,A-Nuclear-Lasso,A-Nuclear-Nuclear
0,200,1,70.105379,62.510102,68.970765,61.567127,61.538592
1,200,1,81.516007,75.776199,82.724515,74.615768,74.600579
2,200,2,84.148007,75.530594,81.894173,73.541975,73.351706
3,200,2,67.495833,59.70207,64.425209,58.10266,58.055605
4,200,3,57.937919,53.056637,57.648407,50.829903,50.843612
5,200,3,64.159991,59.368155,64.542819,56.964438,56.928069
6,200,4,64.376399,58.099287,62.343404,54.849996,54.793526
7,200,4,68.66885,62.404485,66.20291,59.028906,59.004122
8,200,5,69.525891,62.609016,65.637144,58.784551,58.541695
9,200,5,76.592136,69.759261,74.14848,65.251888,65.055451


### Under Setting $\mathcal{A}_h^{\sigma}$

In [149]:
def GenerateCoef_sigma(R : int = 5, s : float=0.50, p1 : int=64, p2 : int=64, h : float=0, K : int=10):
    """
    Parameters
    ----------
    R : int, default = `5`
        The rank of the matrix coef.
    s : float, default = `0.5`
        The sparsity of the matrix coef.
    p1, p2 : int, default = `64`
        THe height and width of the coef.
    h : float, default = `0`
        The deviation level of source domain and target domain.
    K : int, defualt = `10`
        THe number of the imformative source domain.

    Return
    ----------
    B : np.ndarray
        The true coef of target domain with shape `(p1,p2)`
    W : list
        The list of source domians' coef
    """
    p = np.sqrt(1 - (1-s)**(1/R))
    B1 = np.random.binomial(1,p,size=(p1,R))
    B2 = np.random.binomial(1,p,size=(p2,R))
    E = np.identity(R)
    B = B1 @ E @ B2.T
    # generate source domian coef
    W = []
    for k in range(K):
        err = np.random.rand(R)
        err = err / np.linalg.norm(err)
        err = err * (h / (R*np.sqrt(min(p1,p2))))
        D = E + np.diag(err)
        Wk =  B1 @ D @ B2.T
        W.append(Wk.copy())
    return B, D